In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from konlpy.tag import Kkma, Komoran, Okt, Mecab
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from hanspell import spell_checker

In [9]:
df = pd.read_excel('data/token/merged_토큰화_산업군포함_최종.xlsx')

In [7]:
df.set_index('회사', inplace = True)

In [16]:
df.drop(['요약토큰화', '장점토큰화', '단점토큰화', '경영진에게 바라는 점토큰화'], axis = 1, inplace = True)

In [17]:
df.tail()

,회사,날짜,산업군,직무,요약,장점,단점,경영진에게 바라는 점
103365,(주)엘지씨엔에스,2016. 01,IT/웹/통신,IT/인터넷,신입 연봉만 높은 회사였네요 시간 근무를 당연하게 생각하는,신입 교육은 정말 맘에 들었어요 신입 연봉도 다른 기업에 비하면 높고요,야근을 너무 당연하게 생각하고 팀마다 다르지만 야근비를 잘 안 챙겨준 점 내가 왜 ...,전공 무관이지만 전공에 맞는 업무를 배정해줄 순 없었나요 나는 지방 근무를 위해 ...
103366,(주)엘지씨엔에스,2016. 01,IT/웹/통신,마케팅/시장조사,신입들에게는 정말 열심 일 배워 괜찮은 커리어 쌓을 수 있는 곳 경력들 중...,너무 과하지 않고 정도를 걷고 견고하게 짜여 있는 사규가 엄격하지만 그렇기 때문에 ...,일하는 사람만 엄청 일하는 회사 칼퇴근하는 사람은 이상인데 실질적인 영업이...,새로운 사업을 열심 발굴하는 사람 케어에 신경을 써주세요 열심 일만 하는 사람...
103367,(주)엘지씨엔에스,2016. 01,IT/웹/통신,IT/인터넷,업계의 전반적인 부진으로 인하여 설루션 사업 및 해외 사업에 대한 도전을 계속...,회사가 조금씩 기우는 느낌이 있지만 곳곳에 뛰어난 인재들이 산재해 있어서 새로운 발...,오픈마켓에 대한 시장이 점점 안 좋아지고 있어서 기업 매출이 점점 줄고 있음,신규 사업 추진을 선택적으로 선별하여 가능성 있는 소수의 사업분야에 집중 투자했으면...
103368,(주)엘지씨엔에스,2016. 01,IT/웹/통신,IT/인터넷,사업 영역이 넓기 때문에 매년 새로운 업무를 경험해볼 수 있음 비전공자...,프로젝트별로 사람들이 모이기 때문에 수평적인 조직 문화 형성 개인 생활을 존중하며...,프로젝트에 투입 시 업무 강도가 너무 셈 전통 사업은 기반을 잃고 있고 성...,미래 성장 사업으로 설루션 기반 사업을 추진 중인데 선택과 집중을 했으면 함 현...
103369,(주)엘지씨엔에스,2016. 01,IT/웹/통신,IT/인터넷,일도 많고 야근도 많지만 직장의 분위기가 인간적이고 좋음,일은 많지만 분위기는 좋다 육아휴직 휴가를 눈치 안 보고 쓸 수 있다,성과에 따른 인센티브가 거의 없다 매번 새로운 프로젝트로 투입되기 때문에 생활이 ...,야근이나 근태보다 일의 성과중심으로 평가받는 문화가 정착되기를 바란다


In [2]:
# 데이터 불러오기
trade_df = pd.read_excel('data/spelling/유통_무역_운송_토큰화.xlsx')
edu_df = pd.read_excel('교육업.xls')
cons_df = pd.read_excel('건설업.xls')

In [43]:
# 필요 없는 열 삭제
trade_df.drop('Unnamed: 0', axis = 1, inplace = True)
edu_df.drop('Unnamed: 0', axis = 1, inplace = True)
cons_df.drop('Unnamed: 0', axis = 1, inplace = True)

In [4]:
# 회사 열을 인덱스로 설정하기
trade_df.set_index('회사', inplace = True)
edu_df.set_index('회사', inplace = True)
cons_df.set_index('회사', inplace = True)

In [46]:
# 중복된 데이터 삭제
trade_df.drop_duplicates(['요약','장점','단점','경영진에게 바라는 점'], inplace = True)
edu_df.drop_duplicates(['요약','장점','단점','경영진에게 바라는 점'], inplace = True)
cons_df.drop_duplicates(['요약','장점','단점','경영진에게 바라는 점'], inplace = True)

In [48]:
# py-hanspell을 이용한 맞춤법 검사 함수 정의
def spelling(df):
    # [요약]부터 [경영진에게 바라는 점]까지
    for col in df.columns[2:]:
        for j in range(df.shape[0]):
            try:
                # 고칠 게 있으면 고치고 기존 데이터 대체
                result_train = spell_checker.check(df[col][j])
                df[col][j] = result_train.as_dict()['checked']
            except:
                # 없으면 pass
                pass

In [49]:
# 데이터 맞춤법 수정
spelling(trade_df)
spelling(edu_df)
spelling(cons_df)

In [50]:
# 지금까지 전처리 한 결과 엑셀 파일로 저장
trade_df.to_excel(excel_writer = '유통_무역_운송_맞춤법.xlsx')
edu_df.to_excel(excel_writer = '교육업_맞춤법.xlsx')
cons_df.to_excel(excel_writer = '건설업_맞춤법.xlsx')

In [3]:
# 데이터 불러오기
trade_df = pd.read_excel('data/spelling/유통_무역_운송_전처리.xlsx')
edu_df = pd.read_excel('data/spelling/교육업_전처리.xlsx')
cons_df = pd.read_excel('data/spelling/건설업_전처리.xlsx')

In [7]:
# null값 확인
print(trade_df.isnull().sum())
print(edu_df.isnull().sum())
print(cons_df.isnull().sum())

날짜              0
직무              0
요약              0
장점              5
단점             33
경영진에게 바라는 점     5
dtype: int64
날짜               0
직무               0
요약               4
장점               4
단점             108
경영진에게 바라는 점      7
dtype: int64
날짜              0
직무              0
요약              2
장점              3
단점             27
경영진에게 바라는 점    12
dtype: int64


In [6]:
# null값 제거
index = trade_df[trade_df['요약'].isnull()].index
trade_df.drop(index, inplace = True)

In [210]:
# 회사 열을 인덱스로 설정하기
trade_df.set_index('회사', inplace = True)
edu_df.set_index('회사', inplace = True)
cons_df.set_index('회사', inplace = True)

In [211]:
import re

# 특수문자 제거 함수 정의
def clean_text(df):
    for col in df.columns[2:]:
        for j in range(df.shape[0]):
            df[col][j] = re.sub('[-=+,#/\?:^$.@*\"~&%ㆍ!\\‘|\(\)\[\]\<\>`\'…]', ' ', str(df[col][j]))

In [212]:
# 특수문자 제거
clean_text(cons_df)

In [213]:
# 영어 제거 함수 정의
def no_english(df):
    for col in df.columns[2:]:
        df[col] = df[col].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힇]", " ")

In [214]:
# 영어 제거
no_english(cons_df)

In [218]:
# mecab 토큰화 및 불용어 처리
mecab = Mecab()
stopwords = ['은', '는', '이', '가', '도', '다', '의', '한', '에', '게', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '하다', '했다']

cons_df['요약토큰화'] = cons_df['요약'].apply(mecab.morphs)
cons_df['요약토큰화'] = cons_df['요약토큰화'].apply(lambda x: [item for item in x if item not in stopwords])

In [207]:
# Okt 토큰화 및 불용어 처리
okt = Okt()

cons_df['요약토큰화'] = cons_df['요약'].apply(okt.morphs)
cons_df['요약토큰화'] = cons_df['요약토큰화'].apply(lambda x: [item for item in x if item not in stopwords])

In [63]:
# Kkma 토큰화 및 불용어 처리
kkm = Kkma()

cons_df['요약토큰화'] = cons_df['요약'].apply(kkm.morphs)
cons_df['요약토큰화'] = cons_df['요약토큰화'].apply(lambda x: [item for item in x if item not in stopwords])

In [66]:
# Komoran 토큰화 및 불용어 처리
kom = Komoran()

cons_df['요약토큰화'] = cons_df['요약'].apply(kom.morphs)
cons_df['요약토큰화'] = cons_df['요약토큰화'].apply(lambda x: [item for item in x if item not in stopwords])

In [224]:
cons_df.to_excel(excel_writer = '건설업_mecab.xlsx')

In [8]:
# mecab 토큰화 및 불용어 처리
mecab = Mecab()
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

korean_stopwords_path='./stopword_수정.txt'

# 텍스트 파일 오픈
with open(korean_stopwords_path,encoding='utf-8') as f:
    stopwords=f.readlines()
stopwords=[x.strip() for x in stopwords]

In [9]:
trade_df['요약토큰화'] = trade_df['요약'].apply(mecab.morphs)
trade_df['요약토큰화'] = trade_df['요약토큰화'].apply(lambda x: [item for item in x if item not in stopwords])